# Appendix A. Introduction to PyTorch

## The three core components of PyTorch

1. Tensor library
    - Similar to numpy
    - Accelerated computing on GPU, seamless switching between CPU and GPU
2. Auto-diff engine
    - Automatic differentiation
    - Simple and efficient backprop and optimization
3. DL library
    - Modular, flexible, and efficient building blocks
    - Pretrained models
    - Loss functions, optimizers, etc.

In [1]:
import torch
torch.backends.mps.is_available()

True

## Understanding tensors

Default data types: 64-bit integers, 32-bit floats

In [2]:
tensor1d = torch.tensor([1, 2, 3])
print(tensor1d.dtype)

torch.int64


Why 32-bit floats as default
- sufficient precision for most DL tasks
- less memory and compute than 64-bit
- GPU optimized for 32-bit computation

Change the precision

In [3]:
floatvec = tensor1d.to(torch.float32)
print(floatvec.dtype)

torch.float32


`view` is more commonly used than `reshape` in pytorch
- view: requires original data to be contiguous
- reshape: works regardless (copy data if necessary)

`matmul` or `@`

## Seeing models as computation graphs

Example: A logistic regression forward pas

In [4]:
import torch.nn.functional as F # important convention

In [5]:
y = torch.tensor([1.0])
x1 = torch.tensor([1.1])
w1 = torch.tensor([2.2])
b = torch.tensor([0.0])
z = x1 * w1 + b
a = torch.sigmoid(z)
loss = F.binary_cross_entropy(a, y)

## Automatic differentiation made easy

### PARTIAL DERIVATIVES AND GRADIENTS

Computing gradients via autograd (Use chain rule)

In [6]:
from torch.autograd import grad

y = torch.tensor([1.0])
x1 = torch.tensor([1.1])
w1 = torch.tensor([2.2], requires_grad=True)
b = torch.tensor([0.0], requires_grad=True)
z = x1 * w1 + b
a = torch.sigmoid(z)
loss = F.binary_cross_entropy(a, y)

grad_L_w1 = grad(loss, w1, retain_graph=True)
grad_L_b = grad(loss, b, retain_graph=True)

print(grad_L_w1)
print(grad_L_b)

(tensor([-0.0898]),)
(tensor([-0.0817]),)


In [7]:
loss.backward()
print(w1.grad)
print(b.grad)

tensor([-0.0898])
tensor([-0.0817])


## Implementing multilayer neural networks

We can subclass `torch.nn.Module` to implement a neural network
- define the layers in `__init__`
- define how the layers interact in `forward`
- we don't have to implement the backward ourselves

A multilayer perceptron with two hidden layers

In [8]:
class NeuralNetwork(torch.nn.Module):
    def __init__(self, num_inputs, num_outputs):
        super().__init__()
        self.layers = torch.nn.Sequential(
            # layer 1
            torch.nn.Linear(num_inputs, 30),
            torch.nn.ReLU(),
            # layer 2
            torch.nn.Linear(30, 20),
            torch.nn.ReLU(),
            # output layer
            torch.nn.Linear(20, num_outputs),
        )

    def forward(self, x):
        logits = self.layers(x)
        return logits

In [9]:
model = NeuralNetwork(50, 3)
print(model)

NeuralNetwork(
  (layers): Sequential(
    (0): Linear(in_features=50, out_features=30, bias=True)
    (1): ReLU()
    (2): Linear(in_features=30, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=3, bias=True)
  )
)


In [10]:
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Total number of trainable model parameters:", num_params)

Total number of trainable model parameters: 2213


In [11]:
model.layers[0].weight

Parameter containing:
tensor([[-0.0511,  0.1120,  0.0836,  ..., -0.0036, -0.0871,  0.1041],
        [ 0.0786, -0.0852,  0.1052,  ..., -0.1194,  0.0547,  0.0193],
        [-0.0063,  0.1181, -0.0271,  ..., -0.0352, -0.1386, -0.0824],
        ...,
        [ 0.1350, -0.0765,  0.1401,  ..., -0.0542,  0.0245, -0.1267],
        [ 0.0826, -0.0367, -0.0042,  ..., -0.0203,  0.0827,  0.0840],
        [ 0.0073,  0.0513,  0.0342,  ..., -0.0778, -0.0847, -0.1050]],
       requires_grad=True)

In [12]:
model.layers[0].weight.shape

torch.Size([30, 50])

In [13]:
torch.manual_seed(123)
model = NeuralNetwork(50, 3)
print(model.layers[0].weight)

Parameter containing:
tensor([[-0.0577,  0.0047, -0.0702,  ...,  0.0222,  0.1260,  0.0865],
        [ 0.0502,  0.0307,  0.0333,  ...,  0.0951,  0.1134, -0.0297],
        [ 0.1077, -0.1108,  0.0122,  ...,  0.0108, -0.1049, -0.1063],
        ...,
        [-0.0787,  0.1259,  0.0803,  ...,  0.1218,  0.1303, -0.1351],
        [ 0.1359,  0.0175, -0.0673,  ...,  0.0674,  0.0676,  0.1058],
        [ 0.0790,  0.1343, -0.0293,  ...,  0.0344, -0.0971, -0.0509]],
       requires_grad=True)


In [14]:
torch.manual_seed(123)
X = torch.rand((1, 50))
out = model(X)
print(out)

tensor([[-0.1262,  0.1080, -0.1792]], grad_fn=<AddmmBackward0>)


In [15]:
with torch.no_grad():
    out = model(X)
print(out)

tensor([[-0.1262,  0.1080, -0.1792]])


In [16]:
with torch.no_grad():
    out = torch.softmax(model(X), dim=1)
print(out)

tensor([[0.3113, 0.3934, 0.2952]])


## Setting up efficient data loaders

1. Create a custom Dataset class that defines how individual records are loaded
2. Instantiate this class to create different Dataset objects
3. Feed each Dataset object to a data loader by instantiating a DataLoader (Each DataLoader handles data shuffling, batching, etc.)

272-282

Implement a custom `Dataset` class

In [17]:
X_train = torch.tensor(
    [[-1.2, 3.1], [-0.9, 2.9], [-0.5, 2.6], [2.3, -1.1], [2.7, -1.5]]
)
y_train = torch.tensor([0, 0, 0, 1, 1])
X_test = torch.tensor(
    [
        [-0.8, 2.8],
        [2.6, -1.6],
    ]
)
y_test = torch.tensor([0, 1])

In [18]:
from torch.utils.data import Dataset

class ToyDataset(Dataset):
    
    def __init__(self, X, y):
        self.features = X
        self.labels = y
        
    def __getitem__(self, index):
        one_x = self.features[index]
        one_y = self.labels[index]
        return one_x, one_y
    
    def __len__(self):
        return self.labels.shape[0]
    
train_ds = ToyDataset(X_train, y_train)
test_ds = ToyDataset(X_test, y_test)

In [19]:
print(len(train_ds))

5


In [20]:
from torch.utils.data import DataLoader

torch.manual_seed(123)

train_loader = DataLoader(
    dataset=train_ds,
    batch_size=2,
    shuffle=True,
    num_workers=0,
    drop_last=True,
)

test_loader = DataLoader(
    dataset=test_ds,
    batch_size=2,
    shuffle=False,
    num_workers=0
)


In [21]:
for idx, (x, y) in enumerate(train_loader):
    print(f"Batch {idx+1}:", x, y)

Batch 1: tensor([[ 2.3000, -1.1000],
        [-0.9000,  2.9000]]) tensor([1, 0])
Batch 2: tensor([[-1.2000,  3.1000],
        [-0.5000,  2.6000]]) tensor([0, 0])


## A typical training loop

In [22]:
import torch.nn.functional as F

torch.manual_seed(123)
model = NeuralNetwork(2, 2)
optimizer = torch.optim.SGD(model.parameters(), lr=0.5)
num_epochs = 3
for epoch in range(num_epochs):
    model.train()

    for batch_idx, (features, labels) in enumerate(train_loader):
        logits = model(features)
        loss = F.cross_entropy(logits, labels)
        optimizer.zero_grad()  # set grads from previous round to 0
        loss.backward()
        optimizer.step()

        ### logging
        print(
            f"Epoch: {epoch+1:03d}/{num_epochs:03d}"
            f" | Batch {batch_idx:03d}/{len(train_loader):03d}"
            f" | Train Loss: {loss:.2f}"
        )
    model.eval()
    # model eval code

Epoch: 001/003 | Batch 000/002 | Train Loss: 0.75
Epoch: 001/003 | Batch 001/002 | Train Loss: 0.65
Epoch: 002/003 | Batch 000/002 | Train Loss: 0.44
Epoch: 002/003 | Batch 001/002 | Train Loss: 0.13
Epoch: 003/003 | Batch 000/002 | Train Loss: 0.03
Epoch: 003/003 | Batch 001/002 | Train Loss: 0.00


In [23]:
with torch.no_grad():
    outputs = model(X_train)
print(outputs)

tensor([[ 2.8569, -4.1618],
        [ 2.5382, -3.7548],
        [ 2.0944, -3.1820],
        [-1.4814,  1.4816],
        [-1.7176,  1.7342]])


Print class probas

In [24]:
torch.set_printoptions(sci_mode=False)
probas = torch.softmax(outputs, dim=1)
print(probas)

tensor([[    0.9991,     0.0009],
        [    0.9982,     0.0018],
        [    0.9949,     0.0051],
        [    0.0491,     0.9509],
        [    0.0307,     0.9693]])


In [25]:
predictions = torch.argmax(probas, dim=1)
print(predictions)

tensor([0, 0, 0, 1, 1])


To get prediction, no need softmax. We can just argmax to logits

In [26]:
predictions = torch.argmax(outputs, dim=1)
print(predictions)

tensor([0, 0, 0, 1, 1])


In [27]:
torch.sum(predictions == y_train)

tensor(5)

prediction accuracy function

In [28]:
def compute_accuracy(model, dataloader):
    
    model = model.eval()
    correct = 0.0
    total_examples = 0
    
    for idx, (features, labels) in enumerate(dataloader):
        with torch.no_grad():
            logits = model(features)
        
        predictions = torch.argmax(logits, dim=1)
        compare = labels == predictions
        correct += torch.sum(compare)
        total_examples += len(compare)
        
    return (correct / total_examples).item()

In [29]:
print(compute_accuracy(model, train_loader))
print(compute_accuracy(model, test_loader))

1.0
1.0


## Saving and loading models

In [30]:
torch.save(model.state_dict(), "model.pth")

In [31]:
model = NeuralNetwork(2, 2)
model.load_state_dict(torch.load("model.pth"))

/var/folders/bt/p1m1p8ls6rs68x_qsjzsz7qr0000gn/T/ipykernel_99498/44082859.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("model.pth"))


<All keys matched successfully>

- `model = NeuralNetwork(2, 2)` not strictly necessary in the same session
- but we need an instance of model in memory to apply the saved model in general

## Optimizing training performance with GPUs

### PyTorch computations on GPU devices

In [35]:
torch.backends.mps.is_available()

True

In [36]:
tensor_1 = torch.tensor([1., 2., 3.])
tensor_2 = torch.tensor([4., 5., 6.])
print(tensor_1 + tensor_2)

tensor([5., 7., 9.])


In [37]:
tensor_1 = tensor_1.to("mps")
tensor_2 = tensor_2.to("mps")
print(tensor_1 + tensor_2)

tensor([5., 7., 9.], device='mps:0')


### Single-GPU training

In [38]:
torch.manual_seed(123)
model = NeuralNetwork(2, 2)

device = torch.device('mps')
model = model.to(device) # transfer the model to the GPU

optimizer = torch.optim.SGD(model.parameters(), lr=0.5)
num_epochs = 3

for epoch in range(num_epochs):
    
    model.train()
    for batch_idx, (features, labels) in enumerate(train_loader):
        
        features, labels = features.to(device), labels.to(device)
        logits = model(features)
        loss = F.cross_entropy(logits, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        ### LOGGING
        print(f"Epoch: {epoch+1:03d}/{num_epochs:03d}"
        f" | Batch {batch_idx:03d}/{len(train_loader):03d}"
        f" | Train/Val Loss: {loss:.2f}")
    model.eval()

    # Insert optional model evaluation code        
        
        

Epoch: 001/003 | Batch 000/002 | Train/Val Loss: 0.75
Epoch: 001/003 | Batch 001/002 | Train/Val Loss: 0.65
Epoch: 002/003 | Batch 000/002 | Train/Val Loss: 0.44
Epoch: 002/003 | Batch 001/002 | Train/Val Loss: 0.13
Epoch: 003/003 | Batch 000/002 | Train/Val Loss: 0.03
Epoch: 003/003 | Batch 001/002 | Train/Val Loss: 0.00


### Training with multiple GPUs